In [48]:
from bs4 import BeautifulSoup
import requests
import csv
from datetime import date
import DBWriter

#tender platform url, keyword is set manualy for now
url = "https://smarttender.biz/publichni-zakupivli-prozorro/search="
headers = {'User-Agent':'Mozilla/5.0'}

def pageFinder(url, headers, key):

    ''' Getting web source and finding out how many pages are there '''

    #get raw smarttender page
    webPage = requests.get(url+key, headers=headers)

    #create object to parse web page
    soup = BeautifulSoup(webPage.text, "html.parser")

    #finding page links and count of pages
    return soup.find('div', attrs = {'id' : 'MainContent_MainContent_MainContent_Pager'}).get_text().strip()

def getOwnerID(url, headers):
    
    ''' Getting owner ID '''
    
    ownerPage = requests.get(url, headers=headers)
    soup = BeautifulSoup(ownerPage.text, "html.parser")  
    detailedOwnerPage = requests.get('https://Smarttender.biz' + soup.find('iframe')['src'], headers=headers)
    soup2 = BeautifulSoup(detailedOwnerPage.text, "html.parser")
    return int(soup2.find('span', class_ = "info_usreou").get_text())

#setup keyword
key = 'інтернет'

#count of pages in search result
pages = pageFinder(url, headers, key)

#List of tenderDict
tenderList = []

for page in pages:
    
    #create link with current page number
    webPage = requests.get(url+key+f'?p={page}', headers=headers)
    
    #soup objct to parse web page
    soup = BeautifulSoup(webPage.text, 'html.parser')

    #find tender data div (with all needed data)
    tenders = soup.find('div', class_ = 'tenders')
    
    for headOfTable in tenders.find_all('tr', class_ = "head"):

        #tender deadline
        try:
            timeStartEnd = []
            deadlines = headOfTable.find('td', class_ ="col6")
            for deadline in deadlines.find_all('div'):
                timeStartEnd.append(deadline.get_text().strip())
                
        except:
            timeStartEnd = ['No data', 'No data']
        
        if (len(timeStartEnd) == 0):
            timeStartEnd = ['No data', 'No data']

        #tender number
        tenderNumber = headOfTable.find('td', class_ = 'col1').get_text().strip()

        #tender details
        tenderPageLink = headOfTable.find('a', class_ = 'linkSubjTrading')['href']

        #tender name
        tenderName = headOfTable.find('a', class_ = 'linkSubjTrading').get_text().strip()

        #tender price
        tenderPrice = headOfTable.find('span', class_ = "InitRate").get_text().strip()

        #tender host
        tenderOwner = headOfTable.find('a', class_ ="organizer-popover").get_text().strip()

#         #tender owner details
#         tenderOwnerAcc = headOfTable.find('div', class_ = "hidden").get_text().strip() + ' https://smarttender.biz' + headOfTable.find('a', class_ = "organizer-popover", target="_blank")['href']

        #tender uploaded
        tenderStart = timeStartEnd[0]

        #submit till
        tenderEnd = timeStartEnd[1]

        #Dict with tender information
        tender = {
            'tenderNumber': tenderNumber,
            'tenderPageLink': tenderPageLink,
            'tenderName' : tenderName,
            'tenderPrice' : tenderPrice,
            'tenderOwner' : tenderOwner,
            'tenderOwnerID' : getOwnerID(tenderPageLink, headers),
            'tenderStart' : tenderStart,
            'tenderEnd' : tenderEnd
        }

        tenderList.append(tender)

#current date 
today = date.today()

#create csv file with output 
with open(f'output-{today}.csv', 'w') as csvfile: # , encoding="windows-1251"
    csvWriter = csv.writer(csvfile) #, delimiter=';'
    csvWriter.writerow(['Норем тендеру', 'Дата публікації', 'Дата подачі пропозицій', 'Назва', 'Організатор', 'Код ЄДРПОУ', 'Вартість', 'Посилання'])
    for tender in tenderList:
        csvWriter.writerow([tender['tenderNumber'], tender['tenderStart'], tender['tenderEnd'], tender['tenderName'], tender['tenderOwner'], tender['tenderOwnerID'], tender['tenderPrice'], tender['tenderPageLink']])



print('DONE')

DONE


In [52]:
#read from csv
with open(f'output-{today}.csv', encoding='utf-8') as csvfile:
    csvReader = csv.reader(csvfile)
    for row in csvReader:
        print(row)

['Норем тендеру', 'Дата публікації', 'Дата подачі пропозицій', 'Назва', 'Організатор', 'Код ЄДРПОУ', 'Вартість', 'Посилання']
['2147499', 'Опубліковано: 19.02.2018', 'Прийом пропозицій до: 07.03.2018\xa019:00', 'Послуги надання доступу до всесвітньої мережі Інтернет (63 лота)', 'ДП "Київська міська дирекція УДППЗ "Укрпошта""', '1189979', '1\xa0022\xa0943.6\xa0грн.\xa0з\xa0ПДВ', 'https://Smarttender.biz/publichni-zakupivli-prozorro/3508562/']
['2135676', 'Опубліковано: 16.02.2018', 'Прийом пропозицій до: 05.03.2018\xa017:00', 'ДК 021:2015-72411000-4 Постачальники інтернет-послуг (Послуги доступу до міжнародної комп’ютерної мережі )', 'Державне підприємство "СЕТАМ" Міністерства юстиції України', '39958500', '248\xa0830\xa0грн.\xa0з\xa0ПДВ', 'https://Smarttender.biz/publichni-zakupivli-prozorro/3488592/']
['2176583', 'Опубліковано: 26.02.2018', 'Прийом пропозицій до: 28.02.2018\xa018:00', 'Послуги інтернету', 'Київський міський Палац ветеранів', '23731686', '7\xa0100\xa0грн.\xa0з\xa0ПДВ',

In [29]:
DBWriter.DBInsert(tenderList[0])


UnicodeEncodeError: 'ascii' codec can't encode characters in position 193-199: ordinal not in range(128)

In [62]:
import cx_Oracle
import json
from pprint import pprint

server = json.load(open('credentials'))
conndata = f"{server['user']}/{server['pass']}@{server['ip']}:{server['port']}/{server['SID']}"

def priceCleanUp(price):
    
    numbers = '0123456789.'
    result = ''
    for i in price:
        if (i in numbers):
            result+=i
    return float(result)

def Insert(tender):
    
    #input is the DICT with tender information
    
    #init DB connection
    db = None
    
    #converting data types

    id_tender = int(tender['tenderNumber'])
    print(id_tender)
    dt_start = tender['tenderStart'].split(' ')[-1]
    print(dt_start)
    dt_end = tender['tenderEnd'].split(' ')[-1][:10]
    print(dt_end)
    name_tender = tender['tenderName'].strip()
    print(name_tender)
    host_tender = tender['tenderOwner']
    print(host_tender)
    edrpou = tender['tenderOwnerID']
    print(edrpou)
    price = priceCleanUp(tender['tenderPrice'].split('грн')[0])
    print(price)
    link_tender = tender['tenderPageLink']
    print(link_tender)
    
    try:
        db = cx_Oracle.connect(conndata)

        try:
            curs = db.cursor()
            curs.execute(f"""
            
            INSERT INTO {server['table']}
            (id_tender, dt_start, dt_end) 
            VALUES 
            ({id_tender}, TO_DATE('{dt_start}', 'DD.MM.YYYY'), TO_DATE('{dt_end}', 'DD.MM.YYYY')) """)
            
            #test output
            print('################################################')
            curs.execute(f"SELECT * FROM {server['table']}")
            
            pprint(curs.fetchall())
        finally:
            curs.close()
            
    #close DB connection        
    finally:
        if db is not None:
            db.close()
            
print('\n INPUT \n')            
pprint(tenderList[0])
print('#############################################################')
print('\n CLEARED DATA \n')
Insert(tenderList[0])


 INPUT 

{'tenderEnd': 'Прийом пропозицій до: 07.03.2018\xa019:00',
 'tenderName': 'Послуги надання доступу до всесвітньої мережі Інтернет (63 '
               'лота)',
 'tenderNumber': '2147499',
 'tenderOwner': 'ДП "Київська міська дирекція УДППЗ "Укрпошта""',
 'tenderOwnerID': 1189979,
 'tenderPageLink': 'https://Smarttender.biz/publichni-zakupivli-prozorro/3508562/',
 'tenderPrice': '1\xa0022\xa0943.6\xa0грн.\xa0з\xa0ПДВ',
 'tenderStart': 'Опубліковано: 19.02.2018'}
#############################################################

 CLEARED DATA 

2147499
19.02.2018
07.03.2018
Послуги надання доступу до всесвітньої мережі Інтернет (63 лота)
ДП "Київська міська дирекція УДППЗ "Укрпошта""
1189979
1022943.6
https://Smarttender.biz/publichni-zakupivli-prozorro/3508562/
################################################
[('c0a8f865161dc1f945b10000082c2180',
  2147499,
  datetime.datetime(2018, 2, 19, 0, 0),
  datetime.datetime(2018, 3, 7, 0, 0),
  None,
  None,
  None,
  None,
  None,
  date